<a href="https://colab.research.google.com/github/aliaghababaee/SLO_Asieh/blob/main/Optuna_merged_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
!unzip /content/drive/MyDrive/merged_data_final2.zip

Archive:  /content/drive/MyDrive/merged_data_final2.zip
 extracting: test_merged.pkl         
 extracting: test_merged_label.pkl   
 extracting: train_merged.pkl        
 extracting: train_merged_label.pkl  


In [3]:
!unzip /content/drive/MyDrive/new_test_merged_data.zip

Archive:  /content/drive/MyDrive/new_test_merged_data.zip
replace test_merged.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: test_merged.pkl         
replace test_merged_label.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: test_merged_label.pkl   


In [4]:
!pip install optuna
!pip install optuna-integration
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.3 MB/s eta 0:00:00


In [7]:
### SLO model
from tensorflow.keras.models import Model
slo_model =  tf.keras.applications.ResNet101(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3),
    )

for layer in slo_model.layers:
    layer._name = 'SLO_' + layer.name

new_model = Model(inputs=slo_model.input, outputs=slo_model.output)

new_model.save('slo_model_resnet101.h5')

171446536/171446536 [==============================] - 7s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
####################### state of the arts ##############################
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
from keras.layers import Input

def classifier (trial):

    ### SLO model
    slo_model = load_model('slo_model_resnet101.h5')

    for layer in slo_model.layers:
        layer.trainable = False

    ### OCT model

    oct_model =  tf.keras.applications.ResNet101(
        weights='imagenet',
        include_top=False,
        input_shape=(60,256,3),
        )

    for layer in oct_model.layers:
        layer.trainable = False


    inputs_slo = tf.keras.layers.Input((128,128,3))
    slo_output = tf.keras.layers.Flatten()(slo_model(inputs_slo))


    inputs_oct = tf.keras.layers.Input((60,256,3))
    oct_output = tf.keras.layers.Flatten()(oct_model(inputs_oct))


    model = tf.keras.layers.Concatenate(axis=-1)([slo_output, oct_output])


    dropout_l0 = trial.suggest_float("dropout_l0", 0, 0.7,step=0.1)

    model = tf.keras.layers.Dropout(dropout_l0)(model)


    n_layers = trial.suggest_int("n_layers", 1, 5)


    for i in range(n_layers):

        n_units = trial.suggest_int("n_units_l{}".format(i), 8, 4096, log = True)

        model = tf.keras.layers.Dense(n_units, activation = 'relu')(model)

        dropout = trial.suggest_float("dropout_l{}".format (i+1), 0, 0.7,step=0.1)

        model = tf.keras.layers.Dropout(rate = dropout)(model)


    outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(model)

    lr = trial.suggest_float ('lr', 1e-5, 1e-3, log = True)

    my_optimizer = tf.keras.optimizers.Adam (learning_rate= lr)

    model_merged = tf.keras.Model([inputs_slo, inputs_oct] , outputs)


    model_merged.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics='accuracy')

    return model_merged

In [8]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold (n_splits = 5, shuffle = True, random_state = 42)
import keras
import pickle
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def preparing(x, y):

    data_slo  = []
    data_oct  = []
    label     = []
    for i in x:
      for j in range(len(x[i])):
          data_slo.append(np.array(x[i][j][0])/255)
          data_oct.append(np.array(x[i][j][1])*255)
          label.append(y[i])
    data_slo = np.reshape(data_slo, np.shape(data_slo))
    data_oct = np.reshape(data_oct, np.shape(data_oct))
    return data_slo, data_oct, np.array(label)



def objective (trial):

  keras.backend.clear_session()

##################??????????????????????????????????????????????


  datagen_oct = ImageDataGenerator(
  rotation_range= 5, # rotation
  zoom_range= 0.2,
  vertical_flip= True , # vertical fli
  fill_mode='nearest',
  data_format='channels_last',
  #cval=0,
    )

  datagen_slo = ImageDataGenerator(
  rotation_range= 5, # rotation
  width_shift_range= [-30, 30], # horizontal shift
  height_shift_range= [-5, 5] , # vertical shift
  zoom_range= 0.2,
  vertical_flip= True , # vertical flip
  brightness_range= [0.2, 1.5]
    )


  images_train = pickle.load(open("/content/train_merged.pkl", 'rb'))
  labels_train = pickle.load(open("/content/train_merged_label.pkl", 'rb'))

  train_index, val_index = next (skf.split (images_train, list(labels_train.values())))

  x_train = {i: images_train[list(images_train.keys())[i]] for i in train_index}
  x_valid = {i: images_train[list(images_train.keys())[i]] for i in val_index}

  y_trainn = {i: labels_train[list(labels_train.keys())[i]] for i in train_index}
  y_validd = {i: labels_train[list(labels_train.keys())[i]] for i in val_index}


  ################## preparing

  x_train_slo, x_train_oct, y_train = preparing(x_train,y_trainn)
  x_valid_slo, x_valid_oct, y_valid = preparing(x_valid,y_validd)

  ################# Augmentation

  ############ slo
  x_train_aug =np.zeros_like(x_train_slo, dtype=np.float32)

  y_train_aug =np.zeros_like(y_train, dtype=np.float32)


  for i in range(len(x_train_slo)):

    x1= x_train_slo[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen_slo.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    x_train_aug [i,:,:,:] = x.next()

    y_train_aug [i] = y_train[i]

  x_train_slo = np.concatenate ((x_train_slo, x_train_aug), axis=0)

  y_train_slo = np.concatenate ((y_train, y_train_aug), axis = 0)

  ############ OCT
  x_train_aug =np.zeros_like(x_train_oct, dtype=np.float32)

  y_train_aug =np.zeros_like(y_train, dtype=np.float32)

  for i in range(len(x_train_oct)):

    x1= x_train_oct[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen_oct.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    x_train_aug [i,:,:,:] = x.next()

    y_train_aug [i] = y_train[i]

  x_train_oct = np.concatenate ((x_train_oct, x_train_aug), axis=0)

  y_train_oct = np.concatenate ((y_train, y_train_aug), axis = 0)


  indices = np.random.permutation (len (x_train_slo))
  x_train_slo = x_train_slo [indices]
  y_train_slo = y_train_slo [indices]

  x_train_oct = x_train_oct [indices]
  y_train_oct = y_train_oct [indices]


  x_train_slo = np.repeat (x_train_slo, repeats = 3, axis = 3)

  x_train_oct = np.repeat (x_train_oct, repeats = 3, axis = 3)

  x_valid_slo = np.repeat (x_valid_slo, repeats = 3, axis = 3)

  x_valid_oct = np.repeat (x_valid_oct, repeats = 3, axis = 3)


  ####################################################################
  # classification via my model
  ####################################################################
  batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64, 128])

  model = classifier (trial)


  # Generate our trial model.


  model.fit([x_train_slo, x_train_oct],
            np.asarray(y_train_oct, dtype=np.float64),
            batch_size= batch_size,
            epochs=50,
            callbacks=[TFKerasPruningCallback(trial, "val_accuracy"), EarlyStopping(patience=10, verbose=1),
            ReduceLROnPlateau(factor=0.1, patience=10, min_lr=1e-6),
            ModelCheckpoint(f'slo_oct.h5', verbose=1, save_best_only=True, save_weights_only=True)],
            validation_data=([x_valid_slo, x_valid_oct] , np.asarray(y_valid, dtype=np.float64)),
            )
  model.load_weights(f'slo_oct.h5')
  score = model.evaluate ([x_valid_slo, x_valid_oct], np.asarray(y_valid, dtype=np.float64), verbose = 1)

  return score [1]


In [9]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())

study.optimize(objective, n_trials=100)

[I 2024-02-28 17:58:48,814] A new study created in memory with name: no-name-30433ebd-0f55-4ee5-a2e2-7c9776982f19


Epoch 1/50
22/22 [==============================] - ETA: 0s - loss: 1.7890 - accuracy: 0.5740
Epoch 1: val_loss improved from inf to 0.59171, saving model to slo_oct.h5
22/22 [==============================] - 25s 365ms/step - loss: 1.7890 - accuracy: 0.5740 - val_loss: 0.5917 - val_accuracy: 0.7674 - lr: 6.9883e-04
Epoch 2/50
21/22 [===========================>..] - ETA: 0s - loss: 0.5684 - accuracy: 0.6964
Epoch 2: val_loss improved from 0.59171 to 0.35990, saving model to slo_oct.h5
22/22 [==============================] - 3s 124ms/step - loss: 0.5711 - accuracy: 0.6923 - val_loss: 0.3599 - val_accuracy: 0.7674 - lr: 6.9883e-04
Epoch 3/50
21/22 [===========================>..] - ETA: 0s - loss: 0.4044 - accuracy: 0.8571
Epoch 3: val_loss improved from 0.35990 to 0.27835, saving model to slo_oct.h5
22/22 [==============================] - 3s 122ms/step - loss: 0.4048 - accuracy: 0.8550 - val_loss: 0.2783 - val_accuracy: 0.9302 - lr: 6.9883e-04
Epoch 4/50
21/22 [======================

[I 2024-02-28 17:59:52,881] Trial 0 finished with value: 1.0 and parameters: {'batch_size': 16, 'dropout_l0': 0.7, 'n_layers': 4, 'n_units_l0': 263, 'dropout_l1': 0.1, 'n_units_l1': 1940, 'dropout_l2': 0.6000000000000001, 'n_units_l2': 832, 'dropout_l3': 0.6000000000000001, 'n_units_l3': 38, 'dropout_l4': 0.4, 'lr': 0.0006988290143139927}. Best is trial 0 with value: 1.0.


Epoch 1/50
43/43 [==============================] - ETA: 0s - loss: 0.6880 - accuracy: 0.5444
Epoch 1: val_loss improved from inf to 0.66056, saving model to slo_oct.h5
43/43 [==============================] - 21s 174ms/step - loss: 0.6880 - accuracy: 0.5444 - val_loss: 0.6606 - val_accuracy: 0.7674 - lr: 6.2385e-05
Epoch 2/50
43/43 [==============================] - ETA: 0s - loss: 0.5675 - accuracy: 0.7870
Epoch 2: val_loss improved from 0.66056 to 0.41025, saving model to slo_oct.h5
43/43 [==============================] - 4s 89ms/step - loss: 0.5675 - accuracy: 0.7870 - val_loss: 0.4102 - val_accuracy: 0.8140 - lr: 6.2385e-05
Epoch 3/50
43/43 [==============================] - ETA: 0s - loss: 0.2556 - accuracy: 0.9290
Epoch 3: val_loss improved from 0.41025 to 0.30589, saving model to slo_oct.h5
43/43 [==============================] - 3s 66ms/step - loss: 0.2556 - accuracy: 0.9290 - val_loss: 0.3059 - val_accuracy: 0.8605 - lr: 6.2385e-05
Epoch 4/50
43/43 [========================

[I 2024-02-28 18:01:17,429] Trial 1 finished with value: 1.0 and parameters: {'batch_size': 8, 'dropout_l0': 0.6000000000000001, 'n_layers': 4, 'n_units_l0': 8, 'dropout_l1': 0.0, 'n_units_l1': 1487, 'dropout_l2': 0.1, 'n_units_l2': 58, 'dropout_l3': 0.5, 'n_units_l3': 659, 'dropout_l4': 0.4, 'lr': 6.238530466572202e-05}. Best is trial 0 with value: 1.0.


...

Epoch 1/50
5/6 [========================>.....] - ETA: 0s - loss: 0.7008 - accuracy: 0.4750

[I 2024-02-28 18:16:20,989] Trial 17 pruned. Trial was pruned at epoch 0.


Epoch 1/50
21/22 [===========================>..] - ETA: 0s - loss: 0.6835 - accuracy: 0.5446

[I 2024-02-28 18:16:52,951] Trial 18 pruned. Trial was pruned at epoch 0.
[W 2024-02-28 18:17:11,212] Trial 19 failed with parameters: {'batch_size': 8} because of the following error: ResourceExhaustedError().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-8-46740449cf6f>", line 138, in objective
    model = classifier (trial)
  File "<ipython-input-6-ce5f315af69b>", line 18, in classifier
    oct_model =  tf.keras.applications.ResNet101(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/applications/resnet.py", line 556, in ResNet101
    return ResNet(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/applications/resnet.py", line 192, in ResNet
    x = stack_fn(x)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/applications/resnet.py", line 554, in stack_fn
    return stack1(x, 512, 3, name="conv5")
  File "/usr/lo

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 

In [11]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  16
  Number of pruned trials:  2
  Number of complete trials:  13
Best trial:
  Value:  1.0
  Params: 
    batch_size: 64
    dropout_l0: 0.5
    n_layers: 3
    n_units_l0: 1879
    dropout_l1: 0.2
    n_units_l1: 959
    dropout_l2: 0.4
    n_units_l2: 334
    dropout_l3: 0.4
    lr: 0.00014194114248164774
